In [1]:
### Note, crispr pools
# these are over time

In [2]:
import gzip
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path
import regex as re

def open_gzip_optional(fname,mode="rt"):
    if str(fname).endswith("gz"):
        return gzip.open(fname,mode)
    elif str(fname).endswith("bz2"):
        return bz2.open(fname,mode)
    else:
        return open(fname,mode)
            
def reverse_complement(dna):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return ''.join([complement[base] for base in dna[::-1]])

#rootdir = Path("/corgi/otherdataset/ellenbushell/phit/")
rootdir = Path("/corgi/otherdataset/ellenbushell/2023march_phit/")
fastqdir = rootdir / "fastq"

################ Read barcode collection. barcode = sgRNA
#import pandas as pd
bc_file = pd.read_csv("/corgi/otherdataset/ellenbushell/2023march_phit/bc.csv",sep="\t")
bc_file.columns = ["mutant","barcode"]
dict_bc2mutant = dict(zip(bc_file["barcode"],bc_file["mutant"]))



################ Prepare parser
flank_regex = re.compile("ATTATT"+"(\w{18,30})"+"GTTT")

################ Process all files
#sample_dict = {}
list_samples = []
list_counts = []
for seq_file in fastqdir.iterdir():
    if str(seq_file).endswith("fastq.gz"):
        
        #if len(list_samples)>5:
        #    break


        ####### Extract sample name
        #P28011_1172_S11_L001_R1_001.fastq.gz
        # => P28011_1172
        #sample = format_filename(seq_file)
        sample = seq_file.name.split("_S")[0]
        
        #Empty counts dictionary
        count_dict = dict(zip(list(bc_file["mutant"]), [0]*bc_file.shape[0]))
        count_dict["_other"] = 0 
        count_dict["_err"] = 0 
        
        #Reverse complement if read2
        do_revcomp = ("_R2_" in str(seq_file))
        
        if "_R1_" in seq_file.name:
            sample = sample+":R1"
        else:
            sample = sample+":R2"
        print(sample)
        
        
        #For all lines
        with open_gzip_optional(seq_file) as f1:
            it1 = iter(f1)
            count = 0
            while True:
                try:
                    
                    #Read one entry in FASTQ
                    next(it1)
                    theseq = next(it1).strip()
                    next(it1)
                    next(it1)
                    
                    if do_revcomp:
                        theseq = reverse_complement(theseq)

                    #print("> "+theseq)
                        
                    #Parse one entry
                    try:
                        
                        putative_barcode = re.search(flank_regex, theseq)[1]
                        #print(putative_barcode)
                        
                        putative_barcode = putative_barcode  #for this input file
                        
                        if putative_barcode in dict_bc2mutant.keys():
                            count_dict[dict_bc2mutant[putative_barcode]] += 1
                        else:
                            count_dict["_other"] += 1
                            #print(str(count)+" not in dict "+putative_barcode)

                    except TypeError:
                        count_dict["_err"] += 1
                        #print(str(count)+" type err: "+theseq)
                    
                    count=count+1

                except StopIteration:
                    break
        
        #Turn into an array, add to the list
        list_samples.append(sample)
        list_counts.append(
            np.array([count_dict["_other"], count_dict["_err"]]+[count_dict[i] for i in bc_file["mutant"]])
        )
        
        

### Turn it all into one matrix
df = pd.DataFrame(
    np.vstack(list_counts).T,
    columns = list_samples, 
    index = ["_other","_err"]+list(bc_file["mutant"]))

P28657_1001:R1
P28657_1001:R2
P28657_1002:R1
P28657_1002:R2
P28657_1003:R1
P28657_1003:R2
P28657_1004:R1
P28657_1004:R2
P28657_1005:R1
P28657_1005:R2
P28657_1006:R1
P28657_1006:R2
P28657_1007:R1
P28657_1007:R2
P28657_1008:R1
P28657_1008:R2
P28657_1009:R1
P28657_1009:R2
P28657_1010:R1
P28657_1010:R2
P28657_1011:R1
P28657_1011:R2
P28657_1012:R1
P28657_1012:R2
P28657_1013:R1
P28657_1013:R2
P28657_1014:R1
P28657_1014:R2
P28657_1015:R1
P28657_1015:R2
P28657_1016:R1
P28657_1016:R2


In [3]:
df

,P28657_1001:R1,P28657_1001:R2,P28657_1002:R1,P28657_1002:R2,P28657_1003:R1,P28657_1003:R2,P28657_1004:R1,P28657_1004:R2,P28657_1005:R1,P28657_1005:R2,...,P28657_1012:R1,P28657_1012:R2,P28657_1013:R1,P28657_1013:R2,P28657_1014:R1,P28657_1014:R2,P28657_1015:R1,P28657_1015:R2,P28657_1016:R1,P28657_1016:R2
_other,45907,46671,33379,32266,113169,94684,51802,48147,48176,49385,...,69584,87455,21781,24623,25845,32306,21301,23934,29220,36999
_err,8472,6866,103200,102577,41074,37442,30505,29027,15161,13139,...,16317,13293,5021,3754,5666,4669,5785,4816,7024,5831
PBANKA_0211000.1,22506,23802,2497,2636,229028,244123,11681,12211,811,870,...,12,23,12,11,50,54,20,23,87,91
PBANKA_0211000.2,15447,14665,419,385,1092,1032,101,90,12,6,...,17,12,10,8,15,11,10,6,8,5
PBANKA_0515000.1,10758,10438,3902,3811,11207,10978,6029,5766,27573,26820,...,74094,71033,32414,31549,28532,27349,29998,29330,42556,40780
PBANKA_0706400.2,16738,16561,822,809,62,56,4329,4219,14,14,...,24,24,10,13,45,45,40,35,10,11
PBANKA_0706400.1,13261,13432,1209,1213,346,359,1065,1075,14709,15008,...,20,24,14,13,211,214,9,9,10,15
PBANKA_0933700.2,12893,12972,1680,1677,4722,4798,1421,1405,2712,2714,...,846,838,193,190,165,166,222,221,158,152
PBANKA_1013300.1,11963,11752,1010,992,1514,1493,1026,998,4033,3911,...,8902,8500,3529,3453,3989,3778,3872,3801,5024,4797
PBANKA_1013300.2,14882,14647,1879,1830,5523,5519,1246,1212,10152,10020,...,15758,15300,5165,5084,8315,8010,7029,6893,7538,7235


In [4]:
#Check how well it worked
df[0:2]/df.sum(axis=0)

#df[-1]/df.sum(axis=0)



,P28657_1001:R1,P28657_1001:R2,P28657_1002:R1,P28657_1002:R2,P28657_1003:R1,P28657_1003:R2,P28657_1004:R1,P28657_1004:R2,P28657_1005:R1,P28657_1005:R2,...,P28657_1012:R1,P28657_1012:R2,P28657_1013:R1,P28657_1013:R2,P28657_1014:R1,P28657_1014:R2,P28657_1015:R1,P28657_1015:R2,P28657_1016:R1,P28657_1016:R2
_other,0.112507,0.114380,0.130868,0.126505,0.177425,0.148445,0.156539,0.145494,0.136699,0.140130,...,0.110771,0.139220,0.098038,0.110830,0.112898,0.141121,0.092108,0.103493,0.106727,0.135140
_err,0.020763,0.016827,0.404614,0.402171,0.064395,0.058701,0.092182,0.087716,0.043019,0.037282,...,0.025975,0.021161,0.022600,0.016897,0.024751,0.020395,0.025015,0.020825,0.025655,0.021298


In [5]:
df.iloc[ -1]/df.sum(axis=0)


P28657_1001:R1    0.000029
P28657_1001:R2    0.000037
P28657_1002:R1    0.077978
P28657_1002:R2    0.083804
P28657_1003:R1    0.075807
P28657_1003:R2    0.083914
P28657_1004:R1    0.091324
P28657_1004:R2    0.099245
P28657_1005:R1    0.035832
P28657_1005:R2    0.039163
P28657_1006:R1    0.014065
P28657_1006:R2    0.015360
P28657_1007:R1    0.031823
P28657_1007:R2    0.034198
P28657_1008:R1    0.003236
P28657_1008:R2    0.003080
P28657_1009:R1    0.005853
P28657_1009:R2    0.006353
P28657_1010:R1    0.004093
P28657_1010:R2    0.004370
P28657_1011:R1    0.000950
P28657_1011:R2    0.000954
P28657_1012:R1    0.000632
P28657_1012:R2    0.000665
P28657_1013:R1    0.000599
P28657_1013:R2    0.000639
P28657_1014:R1    0.001455
P28657_1014:R2    0.001577
P28657_1015:R1    0.001626
P28657_1015:R2    0.001812
P28657_1016:R1    0.001582
P28657_1016:R2    0.001684
dtype: float64

In [6]:
df.to_csv(rootdir / "counts.csv")

#rootdir = Path("/corgi/otherdataset/ellenbushell/P28011/")
